In this notebook you'll be using [Spotipy](https://github.com/spotipy-dev/spotipy), a Python package, to talk to the Spotify API. This means you won't have to manually create API URLs, you'll just need to figure out how to make Spotipy do it for you! The full Spotipy documentation is available at [https://spotipy.readthedocs.io/](https://spotipy.readthedocs.io/)

# To access *public* Spotify data

You'll want to go to the [Spotify for Developers Dashboard](https://developer.spotify.com/dashboard) and create a new app. This will give you a `client_id` and `client_secret`! It's like a super-advanced version of an API key. When you're setting up your app it will probably also ask you for other things like a redirect URL - just put whatever you want in there, it doesn't matter. If it asks what you want access to, you can pick the Web API (but I don't think it matters).

> The code below won't work since it's *my* secret keys. I've deleted them so that this notebook is nice and safe for me!

In [1]:
# !pip install spotipy --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [spotipy]


In [16]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv
import os

load_dotenv()  # take environment variables

API_SPOTIFY_ID = os.getenv("API_SPOTIFY_ID")
API_SPOTIFY_WP = os.getenv("API_SPOTIFY_WP")

from pprint import pprint

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id=API_SPOTIFY_ID,
    client_secret=API_SPOTIFY_WP,
))

When you want data from Spotify, you can't just go to `/artists/Pixies` in order to get work by Pixies! You have to find a special code for the artist (or song, or album, or whatever). It's called the `uri`.

> You can find more details on searching [on the Spotipy documentation](https://spotipy.readthedocs.io/en/2.22.1/#spotipy.client.Spotify.search) or the [Spotify Web API documentation](https://developer.spotify.com/documentation/web-api/reference/search). Remember that Spotipy is a Python wrapper for the Spotify API, so you don't need to work with any URLs!

To find the `uri`, you first need to do a search. Below we use `sp.search` to search for a particular artist.

In [7]:
# Search for the artist Pixies
results = sp.search(q='artist:Pixies', type='artist')

The `results` it shows us is awful and long and terrible. Instead of showing you how to do that, I already poked through it and found the top artist result from our search.

In [8]:
results['artists']['items'][0]

{'external_urls': {'spotify': 'https://open.spotify.com/artist/6zvul52xwTWzilBZl6BUbT'},
 'followers': {'href': None, 'total': 3410529},
 'genres': ['alternative rock'],
 'href': 'https://api.spotify.com/v1/artists/6zvul52xwTWzilBZl6BUbT',
 'id': '6zvul52xwTWzilBZl6BUbT',
 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5ebcc5ad1ceb002d7bf44218283',
   'height': 640,
   'width': 640},
  {'url': 'https://i.scdn.co/image/ab67616100005174cc5ad1ceb002d7bf44218283',
   'height': 320,
   'width': 320},
  {'url': 'https://i.scdn.co/image/ab6761610000f178cc5ad1ceb002d7bf44218283',
   'height': 160,
   'width': 160}],
 'name': 'Pixies',
 'popularity': 72,
 'type': 'artist',
 'uri': 'spotify:artist:6zvul52xwTWzilBZl6BUbT'}

There we go! The `uri` looks to be `spotify:artist:6zvul52xwTWzilBZl6BUbT`.

Now the sad part: the Spotipy documentation is...... not great. The Spotify Web API docs look good, *but* we're using the Python wrapper, not the raw Spotify API! Luckily Spotipy has a great [list of examples](https://github.com/spotipy-dev/spotipy/tree/master/examples), including one for [an artist's top tracks](https://github.com/spotipy-dev/spotipy/blob/master/examples/simple_artist_top_tracks.py).

```python
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy

lz_uri = 'spotify:artist:36QJpDe2go2KgaRleHCDTp'

client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

results = sp.artist_top_tracks(lz_uri)

for track in results['tracks'][:10]:
    print('track    : ' + track['name'])
    print('audio    : ' + track['preview_url'])
    print('cover art: ' + track['album']['images'][0]['url'])
```

Since we already have the credentials and blah blah blah set up, all we need to do is adapt the `sp.artist_top_tracks(lz_uri)` line and everything below it.

In [9]:
results = sp.artist_top_tracks('spotify:artist:6zvul52xwTWzilBZl6BUbT')

for track in results['tracks'][:10]:
    print(track['name'])

Where Is My Mind? - 2007 Remaster
Here Comes Your Man
All I Think About Now
Hey
Monkey Gone to Heaven
Debaser
Gouge Away
Wave Of Mutilation
The Thing
Gigantic - 2007 Remaster


And that's about it! You use magic codes and it lets you get up-to-date information.

# Your mission

I recently came across a Spotify playlist called [Fall in a 90s Suburb](https://open.spotify.com/playlist/7r2XnAUl6moWkcwOaWgihD?si=505c8f22f4314a33) while researching the band [SEAGULL SCREAMING KISS HER KISS HER](https://open.spotify.com/artist/1WSO9nf7wTj5DZBsncauGr?si=S0xpngxHR1mLF720lMZwxg). The playlist was pretty good, but since since SSKHKH only has like 1,500 listeners each month I was curious about the most/least popular songs on the playlist.

## My questions

1. What are the ten most popular songs on the playlist?
2. What percentage of them have a popularity of zero? Print them out, sorted by the band name.
3. Is popularity relative to the artist, the album, all songs on Spotify, or something else?

### My suggested approach

I suggest approaching this through the following steps:

1. Getting the playlist and print out its **name and description**. 
2. Print out **the name and popularity of each song**
3. Print out **the name, popularity, and artists** of each track on the playlist. Or, if you'd like a shortcut, just pick the first artist.
4. Instead of printing, use these to **create a new dictionary** each time you look at a track. Print out this dictionary. You should be printing out 476 dictionaries!
5. Printing isn't helpful! Instead, after you create the dictionary **append it** to a list called `all_tracks`
6. When you're done, `all_tracks` should have 476 items in it
7. Sort the list by `popularity`, take the **top ten**
8. Filter the list by `popularity`, selecting only the ones with a popularity of `0`

### Tips

**Spotipy documentation:** https://spotipy.readthedocs.io/

**Spotify Web API documentation:** https://developer.spotify.com/documentation/web-api/

- Do this in many, many cells, not all in one!
- You definitely want to [look at the Spotipy examples](https://github.com/spotipy-dev/spotipy/tree/master/examples) to find some good code to base your answer off of. There are a handful that talk about playlists – it might be helpful to read and compare a few of them!
- Getting the playlist name/description is **a different endpoint** than getting the actual songs on the playlist.
- Are you printing out the **same number of tracks as are in the actual playlist?** Take note and be careful! It should be ~476.
- If you're getting the id of playlist songs but not seeing song names, look for `fields='items.track.id,total` in your code. It's only pulling the track's id! Change it to `items.track,total` and it will return [more information about each track](https://developer.spotify.com/documentation/web-api/reference/get-playlists-tracks)
- `all_tracks = []` should be the first line in your cell. That makes sure it always resets to being empty before you start adding tracks to it.
- Be sure the first and last items in `all_tracks` are different – maybe you're accidentally adding the same item each time!
- Normally we sort lists of numbers, which is easy. Sorting a list of dictionaries can be done easily with `key=`. Look it up!
- Pick the most popular 10 songs using list comprehensions
- Filtering is best done with a list comprehension.
- You can sort by things that aren't numbers!

In [ ]:
# "https://open.spotify.com/playlist/7r2XnAUl6moWkcwOaWgihD?si=505c8f22f4314a33&nd=1&dlsi=ba960374c5b64831"

# J'ai une limite de résultats à 100...
playlist_uri = 'spotify:playlist:7r2XnAUl6moWkcwOaWgihD'
results = sp.playlist(playlist_uri)

TypeError: Spotify.playlist() got an unexpected keyword argument 'limit'

In [ ]:
playlist_items(playlist_uri, fields=None, limit=100, offset=0, market=None, additional_types=('track', 'episode'))

In [17]:
pprint(results, indent=4)

{   'collaborative': False,
    'description': 'fuzzy guitars from the 80s, 90s &amp; early 00s for '
                   'feeling angsty as the seasons change.  put on a sweater '
                   'and listen to some indie rock, shoegaze, and noisy twee.',
    'external_urls': {   'spotify': 'https://open.spotify.com/playlist/7r2XnAUl6moWkcwOaWgihD'},
    'followers': {'href': None, 'total': 1799},
    'href': 'https://api.spotify.com/v1/playlists/7r2XnAUl6moWkcwOaWgihD?additional_types=track',
    'id': '7r2XnAUl6moWkcwOaWgihD',
    'images': [   {   'height': None,
                      'url': 'https://image-cdn-ak.spotifycdn.com/image/ab67706c0000da84b9d4216c4d8c3ab45ec70214',
                      'width': None}],
    'name': 'Fall in a 90s Suburb 🍂 ',
    'owner': {   'display_name': 'Chris Smith',
                 'external_urls': {   'spotify': 'https://open.spotify.com/user/1214847370'},
                 'href': 'https://api.spotify.com/v1/users/1214847370',
                 

In [42]:
# the name, popularity, and artists

results['tracks']['items'][0]['track']['name']

results['tracks']['items'][0]['track']['popularity']
results['tracks']['items'][0]['track']['artists'][0]['name']

'Polaris'

In [52]:
playlist_dic = {
    'track_name': [],
    'popularity': [],
    'artists': []
}
for song in results['tracks']['items']:
    name = song['track']['name']
    popularity = song['track']['popularity']
    artists = song['track']['artists'][0]['name']
    # print("____")
    # print(name)
    # print(popularity)
    # print(artists)

    playlist_dic['track_name'].append(name)
    playlist_dic['popularity'].append(popularity)
    playlist_dic['artists'].append(artists)


In [53]:
playlist_dic

{'track_name': ['Waiting For October',
  'Scott Pilgrim',
  'Ginger',
  'Frontwards',
  'First Revival',
  "I Can See It (But I Can't Feel It)",
  'Skyscraper',
  'Jar Of Cardinals',
  'Get Back',
  'Tripoli',
  'Everything Flows',
  "(When You Wake) You're Still in a Dream",
  'Barnaby, Hardly Working',
  'Nail Clinic',
  'Number One Blind',
  'Green Grow The Rushes',
  "Don't Look Back",
  'Sweetness and Light',
  'Marzipan',
  'The Backyard',
  'Box Elder',
  'Polar Bear - 2001 Remaster',
  'Tripping Me Up',
  'Taste - 2001 Remaster',
  'Does This Hurt?',
  'Something More',
  'The Light That Will Cease To Fail',
  'You Have a Light',
  'Gold Star For Robot Boy',
  'Big Sky',
  'My Broken Heart',
  "Gravity's Bringing Us Down",
  'Dreams Burn Down - 2001 Remaster',
  'Cherry Chapstick',
  'All the Umbrellas in London',
  'Into Your Arms',
  'Drown',
  'John Cage Bubblegum',
  "Can't Hardly Wait - The Tim Version",
  'Straight On Home',
  'Fog Over Frisco',
  'Lazy Heart',
  'Rose Pa

In [55]:
import pandas as pd
df = pd.DataFrame(playlist_dic)
df

,track_name,popularity,artists
0,Waiting For October,32,Polaris
1,Scott Pilgrim,48,Plumtree
2,Ginger,31,Lilys
3,Frontwards,0,Pavement
4,First Revival,0,The Amps
...,...,...,...
95,Skip Steps 1 & 3,0,Superchunk
96,Sensitive,30,The Field Mice
97,The Pink,26,Medicine
98,Sleep The Clock Around,0,Belle and Sebastian


# NEXT STEPS